In [1]:
!pwd

/home/shengjiyang/repos/Labs26-Citrics-DS-TeamA/notebooks


In [2]:
import pandas as pd

new_york = pd.read_csv("../data/weather/new_york_ny.csv")

print(new_york.shape)
new_york.head()

(17056, 25)


,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2009-01-01 00:00:00,0,0,0.0,8.7,2,31,11:07 AM,10:50 PM,08:20 AM,...,38,22,62,0.0,1017,-6,10,316,27,10007
1,2009-01-01 06:00:00,0,0,0.0,8.7,2,31,11:07 AM,10:50 PM,08:20 AM,...,33,7,63,0.0,1023,-8,10,315,24,10007
2,2009-01-01 12:00:00,0,0,0.0,8.7,2,31,11:07 AM,10:50 PM,08:20 AM,...,26,4,42,0.0,1025,-3,10,304,23,10007
3,2009-01-01 18:00:00,0,0,0.0,8.7,2,31,11:07 AM,10:50 PM,08:20 AM,...,21,16,49,0.0,1025,-5,10,294,13,10007
4,2009-01-02 00:00:00,0,0,0.1,7.0,2,38,11:28 AM,11:54 PM,08:20 AM,...,14,46,49,0.0,1023,-6,10,260,8,10007


In [3]:
new_york.columns

Index(['date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour',
       'uvIndex', 'moon_illumination', 'moonrise', 'moonset', 'sunrise',
       'sunset', 'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC',
       'WindGustKmph', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'tempC', 'visibility', 'winddirDegree', 'windspeedKmph', 'location'],
      dtype='object')

### Planning

#### Pre-modeling Steps

**Accounting for Seasonal Variations**


- Create dynamic datetime splits separating the four seasons and taking the year as an input,
- Use these datetime splits to subset New York City's data by year and season into smaller DataFrames,
- Identify max, min, avg, and med of the following seasonal weather metrics (both metric and imperial because **AMERICA**):
    - tempC
    - FeelsLikeC, see above
    - totalSnow_cm  **<--**  if None, final app will return a verbal message saying it does not Snow there
    - humidity
    
- Store the above data in a single DataFrame and create visualizations for the annual progression of each


**NOTE:** Cut off the data at the last day of summer 2020 so as to account for the fact that Fall 2020 is not over yet. Go back and make this adjustment to the *weather.py* file.


**Applying the above process to all of the data**


- Encapsulate the above process in a function or functions,
- Devise means of applying the above function(s) to all csv files in the root data/weather directory,
- Organize and store yearly averages in a by_year DataFrame,
- Create visualizations showing how the average seasonal weather of each compares with the average of all,
- Encapsulate the above visualization process into a function for easy use.


**Testing that this process will function in Fast API in notebook**


- Replicate Ryan Herr's notebook 